In [1]:
import torch
from torch import nn, einsum
from einops import rearrange
from typing import Type, Any, Callable, Union, List, Optional
from torch import Tensor

In [2]:
class LambdaLayer(nn.Module):
    def __init__(self, dim, *, dim_k, n = None, r = None, heads = 4, dim_out = None, dim_u = 1):
        super(LambdaLayer, self).__init__()
        assert (dim_out % heads) == 0, 'values dimension must be divisible by number of heads for multi-head query'
        assert (r % 2) == 1, 'Receptive kernel size should be odd'

        dim_out = dim_out if dim_out is not None else dim
        self.u = dim_u
        self.heads = heads

        dim_v = dim_out // heads

        self.to_q = nn.Conv2d(dim, dim_k * heads, 1, bias = False)
        self.to_k = nn.Conv2d(dim, dim_k * dim_u, 1, bias = False)
        self.to_v = nn.Conv2d(dim, dim_v * dim_u, 1, bias = False)

        self.norm_q = nn.BatchNorm2d(dim_k * heads)
        self.norm_v = nn.BatchNorm2d(dim_v * dim_u)

        self.local_contexts = True
        self.pos_conv = nn.Conv3d(dim_u, dim_k, (1, r, r), padding = (0, r // 2, r // 2))


    def forward(self, x):
        b, c, hh, ww, u, h = *x.shape, self.u, self.heads

        q = self.to_q(x)
        k = self.to_k(x)
        v = self.to_v(x)

        q = self.norm_q(q)
        v = self.norm_v(v)

        q = rearrange(q, 'b (h k) hh ww -> b h k (hh ww)', h = h)
        k = rearrange(k, 'b (u k) hh ww -> b u k (hh ww)', u = u)
        v = rearrange(v, 'b (u v) hh ww -> b u v (hh ww)', u = u)

        k = k.softmax(dim=-1)

        λc = einsum('b u k m, b u v m -> b k v', k, v)
        Yc = einsum('b h k n, b k v -> b h v n', q, λc)

        v = rearrange(v, 'b u v (hh ww) -> b u v hh ww', hh = hh, ww = ww)
        λp = self.pos_conv(v)
        Yp = einsum('b h k n, b k v n -> b h v n', q, λp.flatten(3))

        Y = Yc + Yp
        out = rearrange(Y, 'b h v (hh ww) -> b (h v) hh ww', hh = hh, ww = ww)
        return out

layer = LambdaLayer(
    dim = 32,
    dim_out = 32,
    r = 23,         # the receptive field for relative positional encoding (23 x 23)
    dim_k = 16,
    heads = 4,
    dim_u = 4
)

In [3]:
class LambdaBottleneck(nn.Module):
    expansion: int = 4
    def __init__(self, inplanes: int, planes: int, stride: int = 1, downsample: Optional[nn.Module] = None, groups: int = 1, base_width: int = 64, dilation: int = 1, norm_layer: Optional[Callable[..., nn.Module]] = None) :
        super(LambdaBottleneck, self).__init__()
        # Parameter
        self.stride = stride
        width = int(planes * (base_width / 64.)) * groups

        self.conv1 =  nn.Conv2d(inplanes, planes, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(width)
        self.conv2 = LambdaLayer(dim=width, dim_out=width, r = 15, dim_k=16, heads=4, dim_u=4)
        self.bn2 = nn.BatchNorm2d(width)
        self.conv3 = nn.Conv2d(width, planes * self.expansion, kernel_size=1, stride=stride, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)

        # functions
        self.downsample = downsample



    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        
        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [4]:
class LambdaResNet(nn.Module):
    def __init__(self, in_channels=1, base_output=64, layers=[], groups=1, width_per_group= 64, replace_stride_with_dilation=None, zero_init_residual = False, num_classes=1000):
        super(LambdaResNet, self).__init__()
        # Paramter 
        self.inplanes = base_output
        self.dilation = 1
        self.groups = groups
        self.base_width = width_per_group

        if replace_stride_with_dilation is None:
            replace_stride_with_dilation = [False, False, False]

        # Layer
        self.conv_1 = nn.Conv2d(in_channels=in_channels, out_channels=base_output, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn_1 = nn.BatchNorm2d(num_features=base_output)

        self.layer1 = self._make_layer(LambdaBottleneck, 64, layers[0])
        self.layer2 = self._make_layer(LambdaBottleneck, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(LambdaBottleneck, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(LambdaBottleneck, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])

        self.fc = nn.Linear(512 * LambdaBottleneck.expansion, num_classes)


        # Functions
        self.activation = nn.ReLU()
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)


        # init
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, LambdaBottleneck):
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, LambdaBottleneck):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]


    def _make_layer(self, block: LambdaBottleneck, planes: int, blocks: int, stride = 1, dilate = False):
        previous_dilation = self.dilation
        downsample = None

        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion)
            )

        layers = []
        layers.append(LambdaBottleneck(self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, nn.BatchNorm2d))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(LambdaBottleneck(self.inplanes, planes, groups=self.groups, base_width=self.base_width, dilation=self.dilation, norm_layer=nn.BatchNorm2d))

        return nn.Sequential(*layers)
        

    def forward(self, x):
        x = self.conv_1(x)
        x = self.bn_1(x)
        x = self.activation(x)
        x = self.maxpool(x)


        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

In [5]:
batch = torch.rand(2, 3, 512, 512)
model = LambdaResNet(in_channels=3, layers=[3, 4, 6, 3])
model(batch).shape

torch.Size([2, 1000])

In [13]:
%load lambda_rest_net/model/net.py
%load lambda_rest_net/model/lambda_bottleneck.py
%load lambda_rest_net/model/lambda_layer.py
%load lambda_rest_net/model/lambda_res_net.py

In [14]:
from  lambda_rest_net.model.net import Net

ModuleNotFoundError: No module named 'lambda_rest_net.model.lambda_ayer'

In [3]:
model = Net()

NameError: name 'Net' is not defined